# Harrow–Hassidim–Loyd algorithm(HHL) algoritum demo using Q-alchemy's state preparation

HHL algorithm is a quantum algorithm that solves a systems of linear equations. In general, a system of linear $n$-dimensional equations  can be written as follows.

$$
A \vec{x} = \vec{b}
$$

Where $A$ is $n$ x $n$ matrix and $\vec{x}, \vec{b} \in \mathbb{R}^n$. HHL algorithm is proven to provide an exponential speed up given that the $A$ is a sparse, matrix and the condition number $\kappa$ of the system is low.  The condition number $\kappa$ is a measure of how much the output value of the function can change for a small change in the input argument. 

To run the HHL algorithm we first have to map the solution vector $\vec{b}$ to the Hilber space. In this module, we will present how the state preparation algorithm by Q-alchemy can be implemented in this context, and compare the results with IBM's state preparation method.


In [1]:
import os
import sys

os.environ["Q_ALCHEMY_API_KEY"] = "<your key here>"
sys.path.append('..')

In [5]:
import numpy as np
from linear_solvers import NumPyLinearSolver, HHL
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit.quantum_info import Statevector

from q_alchemy.initialize import q_alchemy_as_qasm

In [6]:
def get_solution_vector(solution, n):
    solution_vector = Statevector(solution.state).data[n:n+8].real
    norm = solution.euclidean_norm
    return norm * solution_vector / np.linalg.norm(solution_vector)

## Define matrix $A$ as 8 by 8 Sparse Hermitian matrice
First, we will begin by defining the matrix $A$, and $b$. We will arbitarily define A, and randomly generate corresponding solution vector $\vec{b}$ and normalize it.

In [7]:
A = np.array([[1.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [2.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [3.0, 4.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 5.0, 6.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 0.0, 10.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 12.0]])

# Ensure Hermitian property (conjugate transpose)
A = (A + A.T) / 2.0
#b = [1, 0, 0, 0, 0, 0, 0, 0]
b = np.random.rand(8)
b = b/np.linalg.norm(b) #normalize solution

### Initialize the solution state using Q-alchemy

From the vector $\vec{b}$ we have generated above, we will initialize Q-alchemy and generate a quantum circuit that transforms the quantum state from $|0\rangle$ to $|b\rangle$, where $|b\rangle = \sum_{i=1}^{n} b_i |i\rangle$.

In [8]:
sp_qasm = q_alchemy_as_qasm(b, max_fidelity_loss=0.0)
sp_org = QuantumCircuit.from_qasm_str(sp_qasm)
sp_org.draw(fold=-1)

┌─────────────┐             ┌────────┐  ┌─────────┐┌────────────┐┌─────────┐ ┌────────┐                                                                                                                                            
q_0: ┤ Ry(0.67229) ├─────■───────┤ Rz(-π) ├──┤ Rx(π/2) ├┤ Rz(5.0953) ├┤ Rx(π/2) ├─┤ Rz(3π) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     └─────────────┘   ┌─┴─┐     ├───────┬┘  ├─────────┤├────────────┤├─────────┤┌┴────────┴┐      ┌─────────┐  ┌─────────┐┌────────────┐┌─────────┐  ┌────────┐          ┌────────┐  ┌─────────┐┌────────────┐┌─────────┐  ┌────────┐  
q_1: ──────────────────┤ X ├─────┤ Rz(0) ├───┤ Rx(π/2) ├┤ Rz(5.7387) ├┤ Rx(π/2) ├┤ Rz(7π/2) ├──■───┤ Rz(π/2) ├──┤ Rx(π/2) ├┤ Rz(5.2688) ├┤ Rx(π/2) ├──┤ Rz(2π) ├────■─────┤ Rz(-π) ├──┤ Rx(π/2) ├┤ Rz(5.4482) ├┤ Rx(π/2) ├──┤ Rz(3π) ├──
     ┌─────────────┐┌──┴───┴──┐┌─┴───────┴──┐├─────────┤├────────────┤└─────────┘└──────────┘┌─┴─┐┌┴─────────┴─┐├─────────┤├────────────┤├─────────┤┌─┴────────┴─┐┌─┴─┐┌──┴────────┴─┐├─────────┤├────────────┤├─────────┤┌─┴────────┴─┐
q_2: ┤ Rz(-1.9354) ├┤ Rx(π/2) ├┤ Rz(4.8897) ├┤ Rx(π/2) ├┤ Rz(9.8577) ├───────────────────────┤ X ├┤ Rz(2.5893) ├┤ Rx(π/2) ├┤ Rz(4.5029) ├┤ Rx(π/2) ├┤ Rz(8.2262) ├┤ X ├┤ Rz(0.28243) ├┤ Rx(π/2) ├┤ Rz(4.1344) ├┤ Rx(π/2) ├┤ Rz(8.9365) ├
     └─────────────┘└─────────┘└────────────┘└─────────┘└────────────┘                       └───┘└────────────┘└─────────┘└────────────┘└─────────┘└────────────┘└───┘└─────────────┘└─────────┘└────────────┘└─────────┘└────────────┘

We compare the state preapred by the Q-alchemy, and the actual target state $\vec{b}$.

In [9]:
qc = transpile(sp_org, basis_gates=["id", "rx", "ry", "rz", "cx"])
np.vstack([Statevector(qc).data.real, b])

array([[-0.05492133, -0.35872074, -0.25542591, -0.43444235, -0.49809415,
        -0.26943147, -0.19364888, -0.50609242],
       [ 0.05492133,  0.35872074,  0.25542591,  0.43444235,  0.49809415,
         0.26943147,  0.19364888,  0.50609242]])

In [10]:
num_q1 = qc.num_qubits
qc.draw()

global phase: 0
     ┌─────────────┐               ┌─────────────┐ ┌────────┐     »
q_0: ┤ Ry(0.67229) ├───────■───────┤ Ry(-1.9537) ├─┤ Rz(-π) ├─────»
     └─────────────┘     ┌─┴─┐     └┬────────────┤┌┴────────┤     »
q_1: ────────────────────┤ X ├──────┤ Ry(2.5971) ├┤ Rz(π/2) ├──■──»
      ┌────────────┐┌────┴───┴────┐ └────────────┘└─────────┘┌─┴─┐»
q_2: ─┤ Ry(1.1051) ├┤ Rx(-1.9745) ├──────────────────────────┤ X ├»
      └────────────┘└─────────────┘                          └───┘»
«                                                                     »
«q_0: ────────────────────────────────────────────────────────────────»
«      ┌──────────┐ ┌─────────────┐                    ┌─────────────┐»
«q_1: ─┤ Rz(-π/2) ├─┤ Ry(-2.1272) ├─────────────────■──┤ Ry(-2.3066) ├»
«     ┌┴──────────┴┐└┬────────────┤┌─────────────┐┌─┴─┐├─────────────┤»
«q_2: ┤ Rz(2.5893) ├─┤ Ry(1.3613) ├┤ Rz(-1.1986) ├┤ X ├┤ Rx(0.40375) ├»
«     └────────────┘ └────────────┘└─────────────┘└───┘└─────────────┘»
«                    
«q_0: ───────────────
«        ┌────────┐  
«q_1: ───┤ Rz(-π) ├──
«     ┌──┴────────┴─┐
«q_2: ┤ Ry(0.93461) ├
«     └─────────────┘

### Use IBM's module to construct circuit that performs HHL algorithm

Then, we pass the state preparation circuit created by Q-alchemy to IBM's HHL module, so we can perform the rest of HHL algorithm based on the solution state created by Q-alchemy.

In [11]:
from qiskit_aer import AerProvider
backend = AerProvider().get_backend("statevector_simulator")
backend

StatevectorSimulator('statevector_simulator')

In [34]:
qal_hhl = HHL(quantum_instance=backend).solve(A, qc)

num_q = qal_hhl.state.num_qubits
qal_hhl.state.draw('mpl', style="clifford")

### Comparison of Q-alchemy's solution with IBM's default state preparation method, and classical solution

In this section, we are going to compare the $\vec{x}$ obtained based on Q-alchemy's state, and that obtained based on IBM's default state preparation method and the classical solution.

In [35]:
classical_solution = NumPyLinearSolver().solve(A, b)
hhl_default = HHL(quantum_instance=backend).solve(A, b) #using IBM's default state preparation method

In [36]:
x0 = 2**(num_q-1)

qal_sol = get_solution_vector(qal_hhl, x0)
default_sol = get_solution_vector(hhl_default, x0)
classic_sol = classical_solution.state

In [15]:
np.vstack([qal_sol, default_sol, classic_sol]).T

array([[-0.70133364,  0.70133364,  0.70154128],
       [ 0.54680893, -0.54680893, -0.54660565],
       [-0.14916329,  0.14916329,  0.14886378],
       [ 0.05192695, -0.05192695, -0.05164609],
       [-0.03819749,  0.03819749,  0.03799952],
       [-0.02902809,  0.02902809,  0.02901219],
       [-0.05217233,  0.05217233,  0.05188907],
       [ 0.02985421, -0.02985421, -0.02956744]])

In [16]:
print('Q-alchemy euclidean norm:', qal_hhl.euclidean_norm)
print('IBM euclidean norm:', hhl_default.euclidean_norm)
print('Classical euclidean norm:', classical_solution.euclidean_norm)

Q-alchemy euclidean norm: 0.9064931823969181
IBM euclidean norm: 0.9064931823969188
Classical euclidean norm: 0.906431476126309


### Compare the state prepared by Q alchemy and IBM

In [44]:
hhl_qc = hhl_default.state.data[0].operation.definition
state_IBM = Statevector(hhl_qc).data.real
hhl_qc.draw()

┌───────────┐
q_0: ┤0          ├
     │           │
q_1: ┤1 Isometry ├
     │           │
q_2: ┤2          ├
     └───────────┘

In [45]:
qal_qc = qal_hhl.state.data[0].operation.definition
state_qal = Statevector(hhl_qc).data.real
qal_qc.draw()

global phase: 0
     ┌─────────────┐               ┌─────────────┐ ┌────────┐     »
q_0: ┤ Ry(0.67229) ├───────■───────┤ Ry(-1.9537) ├─┤ Rz(-π) ├─────»
     └─────────────┘     ┌─┴─┐     └┬────────────┤┌┴────────┤     »
q_1: ────────────────────┤ X ├──────┤ Ry(2.5971) ├┤ Rz(π/2) ├──■──»
      ┌────────────┐┌────┴───┴────┐ └────────────┘└─────────┘┌─┴─┐»
q_2: ─┤ Ry(1.1051) ├┤ Rx(-1.9745) ├──────────────────────────┤ X ├»
      └────────────┘└─────────────┘                          └───┘»
«                                                                     »
«q_0: ────────────────────────────────────────────────────────────────»
«      ┌──────────┐ ┌─────────────┐                    ┌─────────────┐»
«q_1: ─┤ Rz(-π/2) ├─┤ Ry(-2.1272) ├─────────────────■──┤ Ry(-2.3066) ├»
«     ┌┴──────────┴┐└┬────────────┤┌─────────────┐┌─┴─┐├─────────────┤»
«q_2: ┤ Rz(2.5893) ├─┤ Ry(1.3613) ├┤ Rz(-1.1986) ├┤ X ├┤ Rx(0.40375) ├»
«     └────────────┘ └────────────┘└─────────────┘└───┘└─────────────┘»
«                    
«q_0: ───────────────
«        ┌────────┐  
«q_1: ───┤ Rz(-π) ├──
«     ┌──┴────────┴─┐
«q_2: ┤ Ry(0.93461) ├
«     └─────────────┘

In [46]:
np.vstack([state_qal[0:8], state_IBM[0:8], b]).T

array([[0.05492133, 0.05492133, 0.05492133],
       [0.35872074, 0.35872074, 0.35872074],
       [0.25542591, 0.25542591, 0.25542591],
       [0.43444235, 0.43444235, 0.43444235],
       [0.49809415, 0.49809415, 0.49809415],
       [0.26943147, 0.26943147, 0.26943147],
       [0.19364888, 0.19364888, 0.19364888],
       [0.50609242, 0.50609242, 0.50609242]])

### Another example with 16 by 16 matrix

In [50]:
from scipy.sparse import csr_matrix

matrix_size = 16
data = np.random.random(matrix_size)
diag = np.diag(data)
upper_triangle = np.triu(np.random.random((matrix_size, matrix_size)))
upper_triangle = upper_triangle + upper_triangle.conj().T 
np.fill_diagonal(upper_triangle, 0) 

full_matrix = diag + upper_triangle

sparse_matrix = csr_matrix(full_matrix)

A = (sparse_matrix.toarray())
b = np.random.rand(matrix_size)
b = b/np.linalg.norm(b) #normalize solution

In [51]:
sp_qasm = q_alchemy_as_qasm(b, max_fidelity_loss=0.0)
sp_org = QuantumCircuit.from_qasm_str(sp_qasm)
sp_org.draw(fold=-1)

┌─────────────┐     ┌────────┐┌─────────┐┌────────────┐┌─────────┐┌────────┐     ┌────────┐  ┌─────────┐  ┌────────────┐ ┌─────────┐  ┌──────────┐                     ┌──────────┐  ┌─────────┐┌────────────┐┌─────────┐ ┌──────────┐        ┌─────────┐  ┌─────────┐┌────────────┐┌─────────┐  ┌────────┐          ┌───────┐   ┌─────────┐┌────────────┐┌─────────┐  ┌────────┐  
q_0: ┤ Ry(0.11305) ├──■──┤ Rz(-π) ├┤ Rx(π/2) ├┤ Rz(3.6033) ├┤ Rx(π/2) ├┤ Rz(3π) ├──■──┤ Rz(-π) ├──┤ Rx(π/2) ├──┤ Rz(3.7106) ├─┤ Rx(π/2) ├──┤ Rz(7π/2) ├────────────────■────┤ Rz(-π/2) ├──┤ Rx(π/2) ├┤ Rz(3.8031) ├┤ Rx(π/2) ├─┤ Rz(5π/2) ├───■────┤ Rz(π/2) ├──┤ Rx(π/2) ├┤ Rz(3.3049) ├┤ Rx(π/2) ├──┤ Rz(3π) ├────■─────┤ Rz(0) ├───┤ Rx(π/2) ├┤ Rz(5.5573) ├┤ Rx(π/2) ├──┤ Rz(3π) ├──
     └─────────────┘┌─┴─┐├────────┤├─────────┤├────────────┤├─────────┤├────────┤  │  └────────┘┌─┴─────────┴─┐└┬─────────┬─┘┌┴─────────┴─┐├─────────┬┘┌────────────┐┌─┴─┐┌─┴──────────┴─┐├─────────┤├────────────┤├─────────┤┌┴──────────┴┐┌─┴─┐ ┌┴─────────┴─┐├─────────┤├───────────┬┘├─────────┤┌─┴────────┴─┐┌─┴─┐┌──┴───────┴──┐├─────────┤├────────────┤├─────────┤┌─┴────────┴─┐
q_1: ───────────────┤ X ├┤ Rz(-π) ├┤ Rx(π/2) ├┤ Rz(3.4946) ├┤ Rx(π/2) ├┤ Rz(3π) ├──┼──────■─────┤ Rz(0.44348) ├─┤ Rx(π/2) ├──┤ Rz(4.2986) ├┤ Rx(π/2) ├─┤ Rz(12.378) ├┤ X ├┤ Rz(-0.37434) ├┤ Rx(π/2) ├┤ Rz(4.5574) ├┤ Rx(π/2) ├┤ Rz(9.0504) ├┤ X ├─┤ Rz(2.0057) ├┤ Rx(π/2) ├┤ Rz(5.082) ├─┤ Rx(π/2) ├┤ Rz(6.4495) ├┤ X ├┤ Rz(-1.5516) ├┤ Rx(π/2) ├┤ Rz(4.3087) ├┤ Rx(π/2) ├┤ Rz(7.8464) ├
                    └───┘└────────┘└─────────┘└────────────┘└─────────┘└────────┘┌─┴─┐    │     └──┬───────┬──┘ ├─────────┤  ├────────────┤├─────────┤ └┬──────────┬┘└───┘└─┬─────────┬──┘├─────────┤├────────────┤├─────────┤└─┬────────┬─┘└───┘ └─┬───────┬──┘├─────────┤├───────────┴┐├─────────┤└─┬────────┬─┘└───┘└─────────────┘└─────────┘└────────────┘└─────────┘└────────────┘
q_2: ────────────────────────────────────────────────────────────────────────────┤ X ├────┼────────┤ Rz(0) ├────┤ Rx(π/2) ├──┤ Rz(3.5943) ├┤ Rx(π/2) ├──┤ Rz(7π/2) ├───■────┤ Rz(π/2) ├───┤ Rx(π/2) ├┤ Rz(5.2341) ├┤ Rx(π/2) ├──┤ Rz(2π) ├────■─────┤ Rz(0) ├───┤ Rx(π/2) ├┤ Rz(5.9361) ├┤ Rx(π/2) ├──┤ Rz(3π) ├────────────────────────────────────────────────────────────────────────
                                                                                 └───┘  ┌─┴─┐    ┌─┴───────┴──┐ ├─────────┤  ├────────────┤├─────────┤ ┌┴──────────┴┐┌─┴─┐ ┌┴─────────┴─┐ ├─────────┤├────────────┤├─────────┤┌─┴────────┴─┐┌─┴─┐┌──┴───────┴──┐├─────────┤├────────────┤├─────────┤┌─┴────────┴─┐                                                                      
q_3: ───────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Rz(1.2609) ├─┤ Rx(π/2) ├──┤ Rz(4.4494) ├┤ Rx(π/2) ├─┤ Rz(8.7428) ├┤ X ├─┤ Rz(2.8295) ├─┤ Rx(π/2) ├┤ Rz(4.4054) ├┤ Rx(π/2) ├┤ Rz(8.2727) ├┤ X ├┤ Rz(0.80093) ├┤ Rx(π/2) ├┤ Rz(3.6783) ├┤ Rx(π/2) ├┤ Rz(8.5486) ├──────────────────────────────────────────────────────────────────────
                                                                                        └───┘    └────────────┘ └─────────┘  └────────────┘└─────────┘ └────────────┘└───┘ └────────────┘ └─────────┘└────────────┘└─────────┘└────────────┘└───┘└─────────────┘└─────────┘└────────────┘└─────────┘└────────────┘

In [55]:
qc = transpile(sp_org, basis_gates=["id", "rx", "ry", "rz", "cx"])
np.vstack([Statevector(qc).data.real, b]).T

array([[-0.14315242,  0.14315242],
       [-0.31421353,  0.31421353],
       [-0.16131349,  0.16131349],
       [-0.02244947,  0.02244947],
       [-0.0505375 ,  0.0505375 ],
       [-0.39551638,  0.39551638],
       [-0.26274936,  0.26274936],
       [-0.24998701,  0.24998701],
       [-0.37172357,  0.37172357],
       [-0.47629726,  0.47629726],
       [-0.18534281,  0.18534281],
       [-0.36988029,  0.36988029],
       [-0.09708925,  0.09708925],
       [-0.04848612,  0.04848612],
       [-0.11927828,  0.11927828],
       [-0.03908738,  0.03908738]])

In [56]:
qal_hhl = HHL(quantum_instance=backend).solve(A, qc)
hhl_default = HHL(quantum_instance=backend).solve(A, b) #using IBM's default state preparation method
classical_solution = NumPyLinearSolver().solve(A, b)

In [61]:
num_q = qal_hhl.state.num_qubits
x0 = 2**(num_q-1)

qal_sol = get_solution_vector(qal_hhl, x0).reshape(-1, 1)
default_sol = get_solution_vector(hhl_default, x0).reshape(-1, 1)
classic_sol = classical_solution.state.reshape(-1, 1)

In [66]:
classic_sol

array([[-0.05042347],
       [ 0.14059217],
       [ 0.37005985],
       [-0.22825135],
       [ 0.55323108],
       [ 0.32747643],
       [-0.39673136],
       [ 0.04280502],
       [-0.31548249],
       [-0.17251163],
       [ 0.07906161],
       [-0.20202862],
       [ 0.31091417],
       [ 0.06876398],
       [ 0.13445587],
       [-0.26804856]])

In [63]:
np.hstack([qal_sol, default_sol, classic_sol]).T

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 8 and the array at index 2 has size 16